<a href="https://colab.research.google.com/github/brian-ho/mde-preterm-2024/blob/main/day_2/notebooks/Calling_an_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# XML to CSV

```
2024 MDE Preterm
Brian Ho - brian@brian-ho.io
Jenny Fan - me@jennyfan.com
```

This notebook is a quick demonstration of how you can call an external Application Programming Interface (API) to fetch data from providers and live sources, rather than just downloading existing files.

If you'd like to incorporate an API into your coursework, let me know! There are some nuanced to setting them up that we can review together.

## About APIs
APIs are made available by a variety of public and private sources, either for free or as a paid offering. One example that we'll use in this notebook are APIs from HERE (https://developer.here.com/) a company that provides geospatial information. Other APIs you might explore include:

- Google Maps Platform (https://mapsplatform.google.com/)
- Mapbox (https://www.mapbox.com/)
- Mapillary (https://www.mapillary.com/developer)
... and more! The internet is truly your oyster — being able to call APIs enables you to access a number of "developer" tools. Of course, that comes with the caveat that things get a little tricker to work with!

## Setting up with HERE
To set up the HERE API, go to https://developer.here.com/ and create your free account. After you do that, you'll need to:

- Create a project in the Project Manger, giving it access to relevant services, e.g. "HERE Search - One Box Search".
0 Create an app in the Access Manager, giving it access to the project you created.
0 You'll use the API key for your app to access data. So long as you stay under 1,000 calls a day, your access wil be free.

In [1]:
# We'll need a new library to work with the encoded geometry information
%pip install flexpolyline --quiet


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
# Don't forget to enter your API key here!
HERE_API_KEY = "YOUR_API_KEY_HERE"

In [41]:
# We'll use requests to call the API and time to manage delays
import requests
import time
from flexpolyline import decode

In [8]:
def search_for_places(latitude, longitude, query):
    """A helper function that calls the HERE Discover API."""

    print(f"Searching for {query} at {latitude}, {longitude} ...")

    # API often limit rate of access — this slows things down
    time.sleep(0.1)

    # Use requests to set query parameters per API documentation
    params = {
        "at": f"{latitude},{longitude}",
        "apiKey": HERE_API_KEY,
        "q": query,
    }

    # Make the request and return response
    return requests.get(
        "https://discover.search.hereapi.com/v1/discover?", params=params
    )

In [29]:
# Try out the API call for 26 Broadway in Manhattan, NYC
places_result = search_for_places(40.70515556422382, -74.01317432574815, "coffee")
places_result_data = places_result.json()

print(f"Found {len(places_result_data['items'])} places!")

Searching for coffee at 40.70515556422382, -74.01317432574815 ...
Found 20 places!


In [30]:
places_result_data["items"][0]

{'title': 'STARBUCKS',
 'id': 'here:pds:place:840dr5re-7f85a287c88549df816dbee3f48f5ff0',
 'language': 'en',
 'ontologyId': 'here:cm:ontology:coffee',
 'resultType': 'place',
 'address': {'label': 'STARBUCKS, 2 Broadway, New York, NY 10004-2207, United States',
  'countryCode': 'USA',
  'countryName': 'United States',
  'stateCode': 'NY',
  'state': 'New York',
  'county': 'New York',
  'city': 'New York',
  'district': 'Financial District',
  'street': 'Broadway',
  'postalCode': '10004-2207',
  'houseNumber': '2'},
 'position': {'lat': 40.70457, 'lng': -74.01313},
 'access': [{'lat': 40.70455, 'lng': -74.01327}],
 'distance': 66,
 'categories': [{'id': '100-1100-0010',
   'name': 'Coffee Shop',
   'primary': True}],
 'chains': [{'id': '1396', 'name': 'STARBUCKS'}],
 'references': [{'supplier': {'id': 'core'}, 'id': '41516220'},
  {'supplier': {'id': 'tripadvisor'}, 'id': '4477956'},
  {'supplier': {'id': 'yelp'}, 'id': 'rPXlTqHLYKcXyADOeqzwbg'}],
 'contacts': [{'phone': [{'value': '+

In [36]:
def search_for_route(
    origin_latitude, origin_longitude, destination_latitude, destination_longitude
):
    """A helper function that calls the HERE Discover API."""

    print(
        f"Searching for route between {origin_latitude}, {origin_longitude} and {destination_latitude}, {destination_longitude} ..."
    )

    # API often limit rate of access — this slows things down
    time.sleep(0.1)

    # Use requests to set query parameters per API documentation
    params = {
        "transportMode": "car",
        "origin": f"{origin_latitude},{origin_longitude}",
        "destination": f"{destination_latitude},{destination_longitude}",
        "return": "polyline",
        "apiKey": HERE_API_KEY,
    }

    # Make the request and return response
    return requests.get("https://router.hereapi.com/v8/routes?", params=params)

In [37]:
routes_results = search_for_route(
    42.363255, -71.126319, 42.36046146521424, -71.05795471680749
)
routes_results_data = routes_results.json()

Searching for route between 42.363255, -71.126319 and 42.36046146521424, -71.05795471680749 ...


In [47]:
polyline = routes_results_data["routes"][0]["sections"][0]["polyline"]

In [48]:
decode(polyline)

[(42.36352, -71.12632),
 (42.36353, -71.1262),
 (42.36353, -71.12616),
 (42.36351, -71.12596),
 (42.36353, -71.12566),
 (42.36354, -71.12553),
 (42.36354, -71.12546),
 (42.3636, -71.12475),
 (42.36361, -71.12464),
 (42.36364, -71.12442),
 (42.36366, -71.12424),
 (42.36368, -71.12405),
 (42.36369, -71.12396),
 (42.3637, -71.12387),
 (42.3637, -71.12379),
 (42.36372, -71.12353),
 (42.36374, -71.12332),
 (42.36375, -71.12311),
 (42.36376, -71.123),
 (42.36378, -71.12278),
 (42.36381, -71.12244),
 (42.36383, -71.12226),
 (42.36385, -71.12203),
 (42.36388, -71.12163),
 (42.36389, -71.12145),
 (42.36391, -71.12128),
 (42.364, -71.12013),
 (42.36402, -71.11985),
 (42.3641, -71.11897),
 (42.36409, -71.11887),
 (42.36408, -71.1188),
 (42.36407, -71.1187),
 (42.36406, -71.11862),
 (42.36405, -71.11856),
 (42.36403, -71.11848),
 (42.36401, -71.11843),
 (42.36398, -71.11838),
 (42.36394, -71.11834),
 (42.36389, -71.11831),
 (42.36384, -71.11829),
 (42.36377, -71.11827),
 (42.36368, -71.11825),
 (4